In [1]:
import numpy as np
import pandas as pd
import vcfpy
import re
from tqdm import tqdm
from pathlib import Path

from pathos.pools import _ProcessPool as Pool
from ncls import NCLS

In [2]:
class GenomePosition():
    genome_pos_pattern = re.compile(r"(.+):(\d+)-(\d+)")

    def __init__(self, chrom, start, end):
        self.chrom = chrom
        self.start = start
        self.end = end

    @classmethod
    def from_str(cls, pos_str):
        match = cls.genome_pos_pattern.match(pos_str)

        if not match:
            return None

        return cls(match[1], int(match[2]) - 1, int(match[3]))

    @classmethod
    def from_vcf_record(cls, record):
        CHROM = record.CHROM.replace("chr", "")
        return cls(CHROM, record.affected_start, record.affected_end)

    @classmethod
    def from_gtf_record(cls, record):
        return cls(record[0].replace("chr", ""), int(record[3]) - 1, int(record[4]))

    def contains(self, other):
        return other.chrom == self.chrom and other.start >= self.start and other.end <= self.end

    def __eq__(self, other):
        return self.chrom == other.chrom and self.start == other.start and self.end == other.end

    def __repr__(self):
        return "%s:%d-%d" % (self.chrom, self.start + 1, self.end)

    def __str__(self):
        return "%s:%d-%d" % (self.chrom, self.start + 1, self.end)

    def __len__(self):
        return self.end - self.start

In [3]:
class GenomeIntervalTree():
    def __init__(self, predicate, records):
        self.predicate = predicate
        self.records = []

        working_tree_map = {}

        idx = 0
        for record in records:
            genome_pos = predicate(record)

            if genome_pos is None:
                continue

            chrom = genome_pos.chrom

            if not chrom in working_tree_map:
                # (starts, ends, ids)
                working_tree_map[chrom] = ([], [], [])

            starts, ends, ids = working_tree_map[chrom]
            starts.append(genome_pos.start)
            ends.append(genome_pos.end)
            ids.append(idx)

            self.records.append(record)
            idx += 1

        tree_map = {}

        for chrom, (starts, ends, ids) in working_tree_map.items():
            tree_map[chrom] = NCLS(
                np.array(starts, dtype=np.long),
                np.array(ends, dtype=np.long),
                np.array(ids, dtype=np.long))

        self.tree_map = tree_map

    def _make_query_params(self, genome_pos_list):
        starts = np.array([genome_pos.start for genome_pos in genome_pos_list])
        ends = np.array([genome_pos.end for genome_pos in genome_pos_list])
        ids = np.array(list(range(len(genome_pos_list))))

        return (starts, ends, ids)

    def _pick_best_record(self, from_ids=None, for_pos=None):
        if len(from_ids) < 1:
            return None

        if len(from_ids) == 1:
            return self.records[from_ids[0]]

        records = [self.records[record_id] for record_id in from_ids]

        scored_records = [(record, self._compute_jaccard_index(for_pos, self.predicate(record))) for record in records]
        sorted_records = sorted(scored_records, key=lambda tup: tup[1], reverse=True)

        return sorted_records[0][0]

    def _compute_jaccard_index(self, pos_a, pos_b):
        if pos_a.chrom != pos_b.chrom:
            return 0

        range_a = range(pos_a.start, pos_a.end)
        range_b = range(pos_b.start, pos_b.end)

        if range_b.start > range_a.stop or range_a.start > range_b.stop:
            return 0

        intersection = range(max(range_a.start, range_b.start), min(range_a.stop, range_b.stop))

        # The following is equivalent to |A ∩ B| / |A ∪ B|, but avoids computing
        # a union.
        # |A ∩ B| / (|A| + |B| + |A ∩ B|)
        return len(intersection) / (len(range_a) + len(range_b) - len(intersection))

    def has_overlap(self, genome_pos):
        tree = self.tree_map.get(genome_pos.chrom)

        if not tree:
            return False

        return tree.has_overlap(genome_pos.start, genome_pos.end)

    def get_first_overlap(self, genome_pos):
        tree = self.tree_map.get(genome_pos.chrom)

        if not tree:
            return None

        qparams = self._make_query_params([genome_pos])
        _, record_ids = tree.first_overlap_both(*qparams)

        if len(record_ids) < 1:
            return None

        return self.records[record_ids[0]]

    def get_best_overlap(self, genome_pos):
        tree = self.tree_map.get(genome_pos.chrom)

        if not tree:
            return None

        qparams = self._make_query_params([genome_pos])
        _, record_ids = tree.all_overlaps_both(*qparams)

        return self._pick_best_record(from_ids=record_ids, for_pos=genome_pos)

    def get_all_overlaps(self, genome_pos):
        tree = self.tree_map.get(genome_pos.chrom)

        if not tree:
            return []

        qparams = self._make_query_params([genome_pos])
        _, record_ids = tree.all_overlaps_both(*qparams)

        return [self.records[record_id] for record_id in record_ids]

    def get_first_containment(self, genome_pos):
        tree = self.tree_map.get(genome_pos.chrom)

        if not tree:
            return None

        qparams = self._make_query_params([genome_pos])
        _, record_ids = tree.all_containments_both(*qparams)

        if len(record_ids) < 1:
            return None

        return self.records[record_ids[0]]

    def get_best_containment(self, genome_pos):
        tree = self.tree_map.get(genome_pos.chrom)

        if not tree:
            return None

        qparams = self._make_query_params([genome_pos])
        _, record_ids = tree.all_containments_both(*qparams)

        return self._pick_best_record(from_ids=record_ids, for_pos=genome_pos)

    def get_all_containments(self, genome_pos):
        tree = self.tree_map.get(genome_pos.chrom)

        if not tree:
            return []

        qparams = self._make_query_params([genome_pos])
        _, record_ids = tree.all_containments_both(*qparams)

        return [self.records[record_id] for record_id in record_ids]

In [27]:
cosmic_path = '../cerebra/wrkdir/CosmicGenomeScreensMutantExport.tsv'
cosmic_df = pd.read_csv(cosmic_path, delimiter='\t')
filtered_db = cosmic_df.loc[cosmic_df["Primary site"] == "lung"]

cosmic_genome_tree = GenomeIntervalTree(
            lambda row: GenomePosition.from_str(str(row["Mutation genome position"])),
            (record for idx, record in filtered_db.iterrows()))

In [29]:
cosmic_genome_tree._pick_best_record

<bound method GenomeIntervalTree._pick_best_record of <__main__.GenomeIntervalTree object at 0x118b9b0d0>>

In [ ]:
#////////////////////////////////////////////////////////////////////
#////////////////////////////////////////////////////////////////////
#////////////////////////////////////////////////////////////////////

In [30]:
class MutationCounter():
    def __init__(self, cosmic_df, hg38_df):
        #filtered_cosmic_df = self._make_filtered_cosmic_df(cosmic_df)
        filtered_cosmic_df = cosmic_df

        self._cosmic_genome_tree = GenomeIntervalTree(
            lambda row: GenomePosition.from_str(str(row["Mutation genome position"])),
            (record for idx, record in filtered_cosmic_df.iterrows()))
        
        self._hg38_genome_tree = GenomeIntervalTree(
            GenomePosition.from_gtf_record,
            (record for idx, record in hg38_df.iterrows()))

In [32]:
cosmic_path = '../cerebra/wrkdir/CosmicGenomeScreensMutantExport.tsv'
ref_anno_path = '../cerebra/wrkdir/hg38-plus.gtf'

cosmic_df = pd.read_csv(cosmic_path, delimiter='\t')
filtered_db = cosmic_df.loc[cosmic_df["Primary site"] == "lung"]
refgenome_df = pd.read_csv(ref_anno_path, delimiter='\t', header=None)

mutation_counter = MutationCounter(cosmic_df, filtered_db)

ValueError: invalid literal for int() with base 10: 'TCGA-05-4427-01'

In [ ]:
mutation_counter